In [3]:
#やりたいこと：株探からEPS,配当,会社名、会社概要、会社URLを取得する。

#情報を取得したい銘柄番号を入力する。
CODEs=[5997,6488,4800]
CSV_FILEPATH='kaisyadat.csv'

###### ブラウザを立ち上げない処理を行うコード  #########################################
from selenium import webdriver              # Webブラウザを自動操作する（python -m pip install selenium)
#import chromedriver_binary                  # パスを通すためのコード
from selenium.webdriver.chrome.options import Options # オプションを使うために必要
option = Options()                          # オプションを用意
option.add_argument('--headless')           # ヘッドレスモードの設定を付与
########################################################################################


import pandas as pd
import time


#################### classの定義 ###########################

class CTRL():

    def __init__(self,dat,CSV_FILEPATH):
    
        self.eps=[]
        self.haitoub=[]
        self.name=[]
        self.gaiyo=[]
        self.url=[]    
        self.CODEs=dat
        self.CSV_FILEPATH=CSV_FILEPATH   

    
    ########## 指定した銘柄の情報を株探からWebスクレイピングにより取得する ###########################
    
    def get_meigarainfo(self):
        i=0  
        browser = webdriver.Chrome(options=option,executable_path="C:/Users/Takeshi/Desktop/work/chromedriver.exe") # Chromeを準備(optionでブラウザ立ち上げ停止にしている）
        for ele in self.CODEs:
            #browser = webdriver.Chrome(options=option,executable_path="C:/Users/Takeshi/Desktop/work/chromedriver.exe") # Chromeを準備(optionでブラウザ立ち上げ停止にしている）
            URL='https://kabutan.jp/stock/?code= '+ str(ele)
            browser.get(URL)  #サイトを開く。ブラウザ自体は立ち上げない

            tmp = browser.find_element_by_class_name('company_block')
            company_name=tmp.find_element_by_tag_name('h3')

            xpath='//*[@id="kobetsu_right"]/div[4]/table/tbody'
            xpath1='//*[@id="kobetsu_right"]/div[3]/table/tbody/tr[3]'

            company_eps=browser.find_element_by_xpath(xpath1 + '/td[4]') #EPS
            company_haitoub=browser.find_element_by_xpath(xpath1 + '/td[5]') #配当
            company_gaiyo=browser.find_element_by_xpath(xpath + '/tr[3]/td') #会社概要
            company_url=browser.find_element_by_xpath(xpath + '/tr[2]/td') #会社URL

            self.eps.append(company_eps.text)
            self.haitoub.append(company_haitoub.text)
            self.name.append(company_name.text)
            self.gaiyo.append(company_gaiyo.text)
            self.url.append(company_url.text)
            i=i+1
            print(str(ele) + ' :finished')
    

        df = pd.DataFrame({'EPS':self.eps, 'HAITOUB':self.haitoub, 'NAME':self.name, 'GAIYO':self.gaiyo, 'URL':self.url},index=CODEs) #データをMatrixに格納する
        df.index.name = "CODE" #index列の名前をCODEにする。
        df.to_csv(self.CSV_FILEPATH) #dfの内容をcsvファイルに書き出す。
        print(df)
        browser.close()
        
    ###################### 銘柄情報を株式投資アプリに入力し、書き込みを行う関数 #####################################
    
    def set_kabudat(self):
         i=0
         matrix= pd.read_csv(self.CSV_FILEPATH) #銘柄情報をcsvファイルから読み取りmatrixに格納する
         print("matrix")            
         print(matrix)
         for i in range(len(matrix['CODE'])):
                
                
             ###########(1)matrixから登録に必要な銘柄情報を読み出している##############################
            
             stock_num=matrix.iloc[i]['CODE'] #matrixより銘柄番号を取得する
             c_name=matrix.iloc[i]['NAME'] #matrixより会社名を取得する
             eps=matrix.iloc[i]['EPS'] #matrixよりepsを取得する            
             url=matrix.iloc[i]['URL'] #matrixより会社WebサイトURLを取得する
             haitoub=matrix.iloc[i]['HAITOUB'] #matrixより配当を取得する     
             kaisya=matrix.iloc[i]['GAIYO'] #matrixより会社概要を取得する。
        
             ###########(2)株式投資アプリを立ち上げ、銘柄情報を自動入力、自動登録を行う関数##############################        
        
             browser = webdriver.Chrome(executable_path="C:/Users/Takeshi/Desktop/work/chromedriver.exe") # Chromeを準備(optionでブラウザ立ち上げ停止に       
             URL='http://25.32.185.252:91/#/'
             browser.get(URL)  #サイトを開く。ブラウザ自体は立ち上げる
            
             elem=browser.find_element_by_id('stock_num') #銘柄番号を入力するフォームを探す
             elem.clear() #フォームの初期値を消去する
             elem.send_keys(int(stock_num))#フォームに銘柄番号を入力する
                                   
             elem=browser.find_element_by_id('c_name') #会社名を入力するフォームを探す
             elem.send_keys(c_name)  #フォームに会社名を入力している  
                        
             elem=browser.find_element_by_id('eps') #EPSを入力するフォームを探す
             elem.send_keys(str(eps))#フォームにEPSを入力している  
                
             elem=browser.find_element_by_id('url')#URLを入力するフォームを探す
             elem.clear()#フォームの初期値を消去する        
             elem.send_keys(url)#フォームにURLを入力している
            
             elem=browser.find_element_by_id('haitoub')#配当を入力するフォームを探す
             elem.send_keys(str(haitoub))#フォームに配当を入力している
            
             elem=browser.find_element_by_id('kaisya')  #会社概要を入力するフォームを探す
             elem.send_keys(kaisya)#フォームに会社概要を入力している
            
             ###########(3) 銘柄情報を登録する##############################               
             elem= browser.find_element_by_id('regdata')#登録を行うコマンドボタンを探す
             elem.click()#コマンドボタンをクリックする
             time.sleep(2)#登録作業が終わるまで待機する(2秒間)
             browser.close()#ブラウザを閉じる
      
            
########## classの定義終了 ##################################################        
            
meigaraget = CTRL(CODEs,CSV_FILEPATH)     #Class CTRLのインスタンス化を行う。
#meigaraget.get_meigarainfo()              #銘柄情報のスクレイピング作業を実施する。
meigaraget.set_kabudat()              #銘柄情報を株式投資アプリに書き込みを行う
print("finish")




matrix
   CODE    EPS  HAITOUB     NAME                                    GAIYO  \
0  5997   73.3     10.0  協立エアテック  空調機器専業中堅。ダンパーで高シェア。冷暖房システムや24時間換気装置も展開。   
1  6488  119.1     36.0     ヨシタケ          自動調整弁専業メーカー。減圧弁、安全弁で首位。海外生産を拡大。   
2  4800   52.9     17.0     オリコン   音楽データベースの老舗。各種ランキング情報を提供。サイト運営や音楽配信等も。   

                           URL  
0    http://www.kak-net.co.jp/  
1  http://www.yoshitake.co.jp/  
2        https://www.oricon.jp  
finish


In [ ]:
#銘柄情報を株式投資アプリに登録する。


class MyClass():
    def __init__(self, message):
        self.value = message
 
myinstance = MyClass("Hello!")
print(myinstance.value)




In [32]:
CSV_FILEPATH='test.csv'
matrix= pd.read_csv(CSV_FILEPATH) 
print("matrix" + "\n")            
print(matrix)
print("\n")
print("matrix1行目抽出"+ "\n") 
print(len(matrix['CODE']))
print("\n")
print("matrix1行目EPS列の要素を抽出"+ "\n") 
print(matrix.iloc[1]['EPS'])

matrix

   CODE   EPS  HAITOUB
0  2152  71.5     14.0
1  7752  85.6     26.0
2  3657  54.0     12.0
3  4238  89.0     20.0


matrix1行目抽出

4


matrix1行目EPS列の要素を抽出

85.6
